In [1]:
%matplotlib inline

In [2]:
path = "data/sample"
#path = "data"

In [3]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions (precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
import utils; reload (utils)
from utils import plots

Using Theano backend.


In [5]:
batch_size=2

In [7]:
import vgg16; reload (vgg16)
from vgg16 import Vgg16